In [113]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from parse import * 
import sqlalchemy as sql
import time 

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
company_df = pd.read_sql_query(query, sql_engine)

df_list = []
for i in range(0, len(company_df)):
    com = company_df.iloc[i]
    com_code = com['종목코드']
    com_name = com['종목명']
    for j in [('D', 'IFRS연결'), ('B', 'KIFRS별도')]:
        query={
            'com_code': com_code, #com_code,
            'report_type': j[0] #D 연결, #B 개별 
                }
        url_tmpl = 'http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{com_code}&cID=&MenuYn=N&ReportGB={report_type}&NewMenuID=105&stkGb=701'
        url = url_tmpl.format(com_code=query['com_code'], report_type=query['report_type'])
        headers={'Referer': url}
        req = requests.get(url, headers=headers).text
        soup = BeautifulSoup(req, 'html.parser')
        table = soup.select('#compBody > div.section.ul_de > div.ul_col2wrap.pd_t25 > div.um_table > table')
        if len(table) != 0:
            df = pd.read_html(str(table[0]).replace('계산에 참여한 계정 펼치기', ''), encoding='utf8')[0]
            df_BOOL = df.isin(['Dividends', 'Multiples', 'FCF']).all(axis = 1)
            df = df.drop([0] + df_BOOL.index[df_BOOL].tolist())
            df = df.set_index([df.columns[0]])
            df_t = df.T

            new_index = {}
            for i in df_t.index:
                year = parse("{}/{}", i)[0]
                year = int(year)
                new_index[i] = year
            df_new = df_t.rename(index=new_index)
            df_new.reset_index(inplace=True)
            df_new = df_new.rename(columns={'index': '년도'})

            df_new.insert(loc=0, column='No', value=None)
            df_new.insert(loc=1, column='종목코드', value=com_code)
            df_new.insert(loc=2, column='종목명', value=com_name)
            df_new.insert(loc=3, column='구분', value='연간')
            df_new.insert(loc=4, column='회계', value=j[1])
            df_new.insert(loc=6, column='분기', value=None)
            df_new.insert(loc=7, column='컨센서스여부', value='실적')
            df_new = df_new.drop([len(df_new)-1])
            df_list.append(df_new)

FNGUIDE_DF = pd.concat(df_list)
FNGUIDE_DF = FNGUIDE_DF.replace(to_replace ="N/A(IFRS)", 
                 value =None) 

No_list = []
for i in range(0, len(FNGUIDE_DF)):
    No_list.append(i+1)
FNGUIDE_DF['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(2),
    '회계': sql.types.VARCHAR(7),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '컨센서스여부': sql.types.VARCHAR(4)
}

for i in list(FNGUIDE_DF.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 

FNGUIDE_DF.to_sql(name="FnGuide_투자지표", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `FnGuide_투자지표` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `FnGuide_투자지표` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `FnGuide_투자지표` ADD INDEX(`종목코드`);')

now = time.localtime()
current_time = "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

print("[" + current_time + "] " + "FnGuide_투자지표 업데이트 완료되었습니다.")

IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
IFRS 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2017/03    Per Share
2018/03    Per Share
2019/03    Per Share
2020/03    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2017/03    Per Share
2018/03    Per Share
2019/03    Per Share
2020/03    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
IFRS 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

...

IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

GAAP 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
GAAP 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
GAAP 연결    Per Share
2016/09    Per Share
2017/09    Per Share
2018/09    Per Share
2019/09    Per Share
2020/06    Per Share
Name: 0, dtype: object
GAAP 개별    Per Share
2016/09    Per Share
2017/09    Per Share
2018/09    Per Share
2019/09    Per Share
2

GAAP 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
GAAP 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/06    Per Share
Name: 0, dtype: object
GAAP 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

IFRS 연결    Per Share
2016/07    Per Share
2017/07    Per Share
2018/07    Per Share
2019/07    Per Share
2020/07    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/07    Per Share
2017/07    Per Share
2018/07    Per Share
2019/07    Per Share
2020/07    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 별도    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 개별    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2020/09    Per Share
Name: 0, dtype: object
IFRS 연결    Per Share
2016/12    Per Share
2017/12    Per Share
2018/12    Per Share
2019/12    Per Share
2

,No,종목코드,종목명,구분,회계,년도,분기,컨센서스여부,EPS(원),EBITDAPS(원),...,FCFF,EPS,BPS,"DPS(보통주,현금)","DPS(1우선주,현금)","배당성향(현금,%)",EBITDAPS,CFPS,SPS,배당성향(현금)
0,1,000020,동화약품,연간,IFRS연결,2016,None,실적,940,824,...,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,000020,동화약품,연간,IFRS연결,2017,None,실적,1683,805,...,228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,000020,동화약품,연간,IFRS연결,2018,None,실적,360,809,...,-51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,000020,동화약품,연간,IFRS연결,2019,None,실적,337,766,...,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5,000020,동화약품,연간,KIFRS별도,2016,None,실적,940,824,...,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,19044,122830,원포유,연간,IFRS연결,2019,None,실적,-123,92,...,-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,19045,122830,원포유,연간,KIFRS별도,2016,None,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19046,122830,원포유,연간,KIFRS별도,2017,None,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19047,122830,원포유,연간,KIFRS별도,2018,None,실적,93,240,...,-93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
